In [1]:
import torch
import torch.nn as nn
import json
import os
import numpy as np
import networkx as nx
import json
import requests
import os
import gzip
import urllib.request
import collections
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import community
import pathlib
from collections import Counter

DATASET STATS


In [2]:
root_dir = "../../../../media/madhav/T7/unarXive/"

In [3]:
from calc_stats import calc_stats
calc_stats(root_dir)

loading previously persisted stats from disk


({'num_paras': array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
         [   0.,    0.,    0., ...,    0.,    0.,    0.],
         [   0.,    0.,    0., ...,    0.,    0.,    0.],
         ...,
         [   0.,    0.,    0., ..., 3195., 1915., 1219.],
         [   0.,    0.,    0., ...,  658., 1068., 1632.],
         [   0.,    0.,    0., ..., 2444.,  337.,  574.]]),
  'num_license_creative_commons': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ..., 14., 11., 10.],
         [ 0.,  0.,  0., ...,  9., 12., 13.],
         [ 0.,  0.,  0., ..., 16.,  4.,  3.]]),
  'num_fig_fails': array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0., ..., 152.,  81.,  82.],
         [  0.,   0.,   0., ...,  50., 118.,  91.],
        

In [15]:
jsonl_fps = []
for path_to_file, subdirs, files in os.walk(root_dir):
        for fn in files:
            fn_base, ext = os.path.splitext(fn)
            if ext == '.jsonl':
                fp = os.path.join(path_to_file, fn)
                if os.path.getsize(fp) > 0:
                    jsonl_fps.append(fp)
print('found {} JSONLs to parse'.format(len(jsonl_fps)))

found 5599 JSONLs to parse


In [16]:
pprs = 0
authors = 0
for fp in jsonl_fps:
        with open(fp) as f:
            for i, line in enumerate(f):
                ppr = json.loads(line)
                authors += len(ppr.get('metadata', {})['authors_parsed'])
print('found {} authors'.format(authors))

found 1288812 authors


In [26]:
from calc_stats import paper_stats
def scrap(jsonl_fps, metric_name: str = None):
    papers = list()
    for fp in jsonl_fps:
        with open(fp) as f:
            for i, line in enumerate(f):
                ppr = json.loads(line)
                metadata = ppr.get('metadata', {})
                
                paper_title = metadata["title"]
                paper_date = metadata['update_date']
                
                authors = metadata['authors_parsed']
                temp = []
                for author in authors:
                    author = " ".join(author)
                    temp.append(author)
                authors = temp
                stats = paper_stats(ppr)

                paper = {'title': f"{paper_title}", 'authors': authors, 'paper_date': paper_date}

                papers.append(paper)
    return papers

In [27]:
def generate_unique_authors(papers: list):
    authors_dict = collections.defaultdict(dict)
    for paper in papers:
        for author in paper['authors']:
            for author2 in paper['authors']:
                if author not in authors_dict or author2 not in authors_dict[author]:
                    authors_dict[author][author2] = 0
                authors_dict[author][author2] += 1

    return authors_dict


In [28]:
def create_matrix(papers: list):
    authors = generate_unique_authors(papers)
    author_matrix = np.zeros([len(authors), len(authors)], dtype=int)
    i = 0
    for key in authors:
        j = 0
        for key2 in authors:
            if key2 in authors[key]:
                author_matrix[i][j] = authors[key][key2]
            j += 1
        i += 1
    df = pd.DataFrame(author_matrix, index=authors.keys(), columns=authors.keys())
    return df

In [29]:
def generate_communities(matrix: pd.DataFrame, papers: list):
    np.fill_diagonal(matrix.values, 0)
    print(matrix)
    adjacency_matrix = matrix.values

    graph_aux = nx.from_numpy_matrix(adjacency_matrix)

    mapping = dict(zip(graph_aux.nodes(), matrix.index))
    graph = nx.relabel_nodes(graph_aux, mapping)
    memberships = list(community.greedy_modularity_communities(graph))

    membership_authors = dict()

    i = 0
    for membership_set in memberships:
        for author in membership_set:
            membership_authors[author] = str(i + 1)
        i = i + 1

    paper_communities = {}
    activity = list()
    for paper in papers:
        paper_communities[paper['title']] = {'communities': set(),'paper_date': str(),
                                             'communities_name': set()}

        for author in paper['authors']:
            paper_communities[paper['title']]['communities'].add(
                membership_authors[author])
            paper_communities[paper['title']]['paper_date'] = paper['paper_date']
        paper_communities[paper['title']]['communities'] = "+".join(paper_communities[paper['title']]['communities'])

        for paper_community in paper_communities[paper['title']]['communities_name']:
            activity.append(paper_community)
    activity_counter = collections.Counter(activity)

    # exports
    df = pd.DataFrame.from_dict(paper_communities, orient="index")
    df3 = pd.DataFrame.from_dict(dict(activity_counter), orient="index")
    df.index.names = ['title']
    df3.index.names = ['institution']
    df3.columns = ['counter']
    df.to_csv('paper-communities.csv')
    df3.to_csv('activity-communities.csv')
    with open("all-papers.json", "w") as outfile:
      json.dump(papers, outfile)

    print(df.head())
    df.reset_index(level=0, inplace=True)
    df3.reset_index(level=0, inplace=True)
    return df, df3

In [30]:
def generate_ranking(dataset: list):
    papers = scrap(dataset)
    matrix = create_matrix(papers)
    return generate_communities(matrix, papers)

In [ ]:
all_papers = []
df1,df3 = generate_ranking(jsonl_fps)
df1.communities_name = df1.communities_name.apply("@".join)
all_papers.append(df1)